In [1]:
from DefSD import *
from astropy.analytic_functions import blackbody
from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel



In [92]:
filename = 'dataArt1SDSSFound'
resultListDT=Table.read(filename+'.tbl', format='ascii')
for x in range(len(resultListDT['ObjID'])):
    name = resultListDT['ObjID'][x]
    try:
        hdulist = fits.open('Exam_fits/'+name+'.fits')
    except IOError:
        print "Error opening the file: ", name

In [93]:
#   a table resultListDT relates the name of the objects and the resulting temoerature and density
#   for the four couples of species

resultListDT=Table.read(filename+'.tbl', format='ascii')
ion = [' [OIII]', ' [SII]', ' [NII]', ' [OII]', ' [NII]', ' [SII]', ' [OIII]', ' [OII]']
for i in range(1,5):
    colT= Column(name='Temp_'+str(i)+ion[2*(i-1)], length=len(resultListDT),unit=u.K)
    colD= Column(name='Den_'+str(i)+ion[2*(i-1)+1], length=len(resultListDT),unit=1 / (u.cm**3))
    resultListDT.add_columns([colT,colD])
resultListDT

#   Obtain the lists Spectra, Headers, spectrum variance of the intensity according with a list of galaxy names
#   and a Table Tabla of models for each line for all the spectra

Headers = []
Spectra = []
Spectra2 = []
SpectraCont = []
Lambda = []
Modelos=[]
Tablas=[]
HDUs=[]
TypeFITS=[]
SpectraIvar = []
LINEProfile = []
paramCont=0.1
typeFITS =0
for x in range(len(resultListDT['ObjID'])):
    try:
        name = resultListDT['ObjID'][x]
        z=resultListDT['z'][x]
        isFits = resultListDT['isFits'][x]
        if isFits:
            hdulist = fits.open('Exam_fits/'+name+'.fits')
            head= hdulist[0].header
            typeFITS=0
            if head['NAXIS'] == 0:
                total= hdulist[1].data
            elif head['NAXIS'] == 1:
                typeFITS=1
            if typeFITS==0:
                spectraFlux = total['flux']
                spectraIvar = total['ivar']
                linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
                lambd = linlam/(1+z)
            else:
                spectraFlux = hdulist[0].data #*(1e-19)
                spectraIvar = spectraFlux*0
                if 'CDELT1' in head:
                    lambd = np.linspace(head['CRVAL1'], head['CRVAL1']+head['CDELT1']*(len(spectraFlux)-1), len(spectraFlux))
                elif 'W0' in head:
                    lambd = np.linspace(head['W0'], head['W0']+head['WPC']*(len(spectraFlux)-1), len(spectraFlux))
                else:
                    print "Can't identify FITS format in file: ", str(name)
                    break
                lambd = lambd/(1+z)
        else:
            specTable = Table.read('Exam_fits/'+str(name)+'.txt', format='ascii')
            spectraFlux = specTable['Intensity'].data #.tolist()
            spectraIvar = spectraFlux*0
            lambd = specTable['Spectral-axis'].data #.tolist()
            lambd = lambd/(1+z)
        orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)

        TypeFITS.append(typeFITS)
        Spectra2.append(spectraFlux)
        print "Object [",x,"]: ", name
        spec =np.array(spectraFlux)
        spec2, lambd = r(spec, lambd, paramCont*orderU)
        
        #gauss_kernel = Gaussian1DKernel(2)
        #smoothed_data_gauss = convolve(spectraFlux, gauss_kernel)
        #spec2 = smoothed_data_gauss

        limResolucion = mt.ceil(lambd[1]-lambd[0])

    # ..............    

        if lambd[0] > 3720.0 :
            print "The object isn't in expected spectrum region (3700 - 6800 Angstrom)"
            ListaLineas=Table.read('LinesPrincipal.txt', format='ascii')
            LS=np.zeros([13,10])

            lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra2, limResolucion)
            LINEProfile.append(lineProf)
            # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
            # que es mas preciso al determinar la amplitud del modelo gaussiano
            for ina in range(len(lineProf)):
                lInf = lineProf[ina]['inf']
                lSup = lineProf[ina]['sup']
                lDel = lSup-lInf
                if (lInf-3*lDel) >= 0 :
                    lInf = lInf-3*lDel
                if (lSup+3*lDel) <= len(lambd) :
                    lSup = lSup+3*lDel
                m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
                lineLambd = lambd[lInf:lSup]
                lineIntCont = lines( lineLambd , m1, b1)
                spec2[lInf:lSup] = lineIntCont
            ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra2, spectraIvar,2000, limResolucion)
    #        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)
        else:
            # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
            # que es mas preciso al determinar la amplitud del modelo gaussiano
            ListaLineas=Table.read('LinesAll.txt', format='ascii')
            lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra2, limResolucion)
            LINEProfile.append(lineProf)
            for ina in range(len(lineProf)):
                lInf = lineProf[ina]['inf']
                lSup = lineProf[ina]['sup']
                lDel = lSup-lInf
                if (lInf-3*lDel) >= 0 :
                    lInf = lInf-3*lDel
                if (lSup+3*lDel) <= len(lambd) :
                    lSup = lSup+3*lDel
                m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
                lineLambd = lambd[lInf:lSup]
                lineIntCont = lines( lineLambd , m1, b1)
                spec2[lInf:lSup] = lineIntCont
            LS=np.zeros([len(ListaLineas),10])
            ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra2, spectraIvar,2000, limResolucion)
    #        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)

        Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','Flux_Err', 'FWHM', 'eqWidth', 'S/N>3'),
                    dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8','S5'))
        Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
        Tabla['Element']=ListaLineas['SPECTRUM']
        AA=split_list(ModelG1[0].parameters, wanted_parts=len(ListaLineas))
        #AAA=split_list(ModelG1[1].parameters, wanted_parts=2)
    #    AA=split_list(ModelL1[0].parameters, wanted_parts=len(ListaLineas))
    #    AAA=split_list(ModelL1[1].parameters, wanted_parts=2)
        for ii in range(len(ListaLineas)):
            if 3*np.mean(spectraIvar[lineProf[0]['inf']:lineProf[0]['sup']]) >= AA[ii][0]:
                Tabla['S/N>3'][ii] = str(False)
            else:
                Tabla['S/N>3'][ii] = str(True)
            if AA[ii][2]<0:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], mt.fabs(AA[ii][2])])
            else:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], AA[ii][2]])
            #if ii==0 or ii==1:
            #    if AAA[ii][2]<0:
            #        Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], mt.fabs(AAA[ii][2])])
            #    else:
            #        Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], AAA[ii][2]])
        Modelos.append(ModelG1)
        #    Modelos.append(ModelL1)
        #print Tabla 
        Tablas.append(Tabla)
        SpectraIvar.append(spectraIvar)
        SpectraCont.append(spec2)
        Lambda.append(lambd)
        Spectra.append(spectraFlux)
    except ValueError:
        print "It has not been possible to process the spectrum of: ", name

Object [ 0 ]:  SDSS J165712.75+321141.4SP1
It has not been possible to fit the line:  4932.603


/usr/lib64/python2.7/site-packages/astropy-1.0rc1-py2.7-linux-x86_64.egg/astropy/modeling/functional_models.py:100: RuntimeWarning: divide by zero encountered in true_divide
  return amplitude * np.exp(- 0.5 * (x - mean) ** 2 / stddev ** 2)


In [95]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 
%matplotlib
### inline
for jj in [0]:  #  range(len(resultListDT)):
#    plt.errorbar(Lambda[jj],Spectra[jj],SpectraIvar[jj],  c='gray',lw=1,label='Spectrum')
    plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
    plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
    lPPP = lineProf
    for iasiao in range(len(lPPP)):
        plt.plot(Lambda[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]],Spectra[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]], lw=2.0, c = 'b')


Using matplotlib backend: Qt4Agg


###     EqWidth ,  FWHM,  FLUX

In [96]:
#   The plot of relative intensity and continuum emission for the galaxy index jj and the wavelenght of all the lines
#   The equivalent width, flux and FWHM for galaxy jj is measured too        
for jj in range(len(Tablas)):
    spe = []
    speCont = []
    for xind in range(len(Spectra[jj])):
        spe.append((Spectra[jj][xind])/(SpectraCont[jj][xind]))
        speCont.append((SpectraCont[jj][xind])/(SpectraCont[jj][xind]))

    lFWHM = []
    eqW = []    ##  lista de valores 
    #  una vez normalizado el espectro observado a la medida del contínuo se integra por el método de Simpson
    print jj
    for iLineProf in range(len(LINEProfile[jj])):
        try:
            f = FWHM(Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], Spectra[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]])
            if len(f) == 0:
                lFWHM.append(0)
            else:
                lFWHM.append(f[0])
        except ValueError:
            lFWHM.append(0)
        sInteg = simps(y=spe[LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], x=Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], even='avg')
        eqW.append(sInteg)
    #   Flux determination for each model in galaxy list
    #   assignment of the FWHM and equivalent width in the Table for the first galaxy
    for i in range(len(Tablas[jj])):
        if Tablas[jj][i]['Amplitude']<=0:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]= 0,0
        else:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]=quad(gaussian,Tablas[jj][i]['Wavelength']-3*Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']+3*Tablas[jj][i]['Sigma'],args=(Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']))
            Tablas[jj]['FWHM'][i],Tablas[jj]['eqWidth'][i] = lFWHM[i], eqW[i]
    Tablas[jj].write('resultTables/Tabla'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
Tablas

0


[<Table masked=False length=16>
   Amplitude      Wavelength      Sigma     ...    eqWidth     S/N>3  
    float64        float64       float64    ...    float64    string40
 -------------- ------------- ------------- ... ------------- --------
  301.080052796 3728.13869534 1.27187255185 ...   179.9174014     True
  412.079318883  3730.9086117 1.11494146519 ...   179.9174014     True
  150.166797468 4342.91573473 1.22001090233 ... 49.1083937157     True
  16.1032921886 4365.56226049 1.35484865886 ... 12.1454267502     True
  312.663720723 4864.20980288 1.26912830687 ... 127.794857111     True
            0.0      4932.603           0.0 ...           0.0    False
  455.273110324 4961.86659134 1.24599350507 ... 144.959833121     True
  1331.92028084 5009.85217174 1.26594398592 ...  442.00972728     True
 0.735222560447  5755.6232304 14.4597394451 ... 4.48205430171     True
 0.834530461158 6532.67554113 3.49259683629 ... 8.00585095334     True
  10.7997258899 6552.03167182 1.90988812343 .

###        Electron density and temperature and plot

In [7]:
#   it obtains the electron density and temperature by the PyNeb cross iterative method for all couple of given species.

fig = plt.figure()
fig.suptitle('')
ax = fig.add_subplot(111)
ax.set_ylabel('Density $n_e (cm^{-3})$')
ax.set_xlabel('Temperature $T_e (K)$')

for jj in range(len(Tablas)):
    k=0
    #if Lambda[jj][0] > 3720.0 :
    #    k=2
    try:
        a=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        b=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
        c=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        d=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
    except ValueError:
        continue

    for i in IntrinsicHB:
        rc.setCorr((Tablas[jj]['Flux'][11-k]/Tablas[jj]['Flux'][4-k])/i, Tablas[jj]['Lambda Vac'][11-k], Tablas[jj]['Lambda Vac'][4-k])
        try:
            a2=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            b2=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
            c2=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])),Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            d2=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
        except ValueError:
            continue
        if b2 != () or d2 != ():
            plt.scatter(b2[0], b2[1],c='c')
            plt.scatter(d2[0], d2[1],c='c')
        if i == 2.86:
            resultListDT['Temp_1 [OIII]'][jj]=a2[0]
            resultListDT['Den_1 [SII]'][jj]=a2[1]
            resultListDT['Temp_3 [NII]'][jj]=c2[0]
            resultListDT['Den_3 [SII]'][jj]=c2[1]
            if b2 != () or d2 != ():
                resultListDT['Temp_2 [NII]'][jj]=b2[0]
                resultListDT['Den_2 [OII]'][jj]=b2[1] 
                resultListDT['Temp_4 [OIII]'][jj]=d2[0]
                resultListDT['Den_4 [OII]'][jj]=d2[1]
            #if Lambda[jj][0] > 3720.0 :
            #    resultListDT['Temp_2 [NII]'][jj]=0.0
            #    resultListDT['Den_2 [OII]'][jj]=0.0
            #    resultListDT['Temp_4 [OIII]'][jj]=0.0
            #    resultListDT['Den_4 [OII]'][jj]=0.0
    resultListDT.write('resultTables/TemDenGalaxies'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
resultListDT


ObjID,ra,dec,z,isFits,Temp_1 [OIII],Den_1 [SII],Temp_2 [NII],Den_2 [OII],Temp_3 [NII],Den_3 [SII],Temp_4 [OIII],Den_4 [OII]
,,,,,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3
string216,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
SDSS J165712.75+321141.4SP1,254.302993774,32.1949996948,0.038,1,12783.4814362,1054.39217232,nan,nan,nan,nan,12792.6619893,339.094734746


###                     Plots the fitting spectrum normalized of index jj

In [21]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 

%matplotlib
jj=1
if isFits:
    if TypeFITS == 0:
        print HDUs[jj][0].header['FLAVOR']
        print HDUs[jj][0].header['QUALITY']
        print HDUs[jj][0].header['FBADPIX'] ## %%Malos Pixeles
        print HDUs[jj][2].data['SN_MEDIAN_ALL']
        print HDUs[jj][2].data['SN_MEDIAN']

lambd=Lambda[jj]

plt.figure(figsize=(16, 9))
plt.title('Spectrum Galaxy '+ resultListDT['ObjID'][jj])
plt.xlabel(r'Wavelength (Angstroms $\AA$) ')
if TypeFITS == 0:
    plt.ylabel(r'$f_{\lambda}$ ('+str(head['BUNIT'])+')')
else:
    plt.ylabel(r'$f_{\lambda}$')

plt.errorbar(Lambda[jj],Spectra[jj]-SpectraCont[jj],SpectraIvar[jj],  c='gray',lw=1,label='Spectrum')
plt.plot(Lambda[jj], Spectra[jj]-SpectraCont[jj], c='k', lw=0.8, label='Model')
plt.plot(Lambda[jj], Modelos[jj][0](Lambda[jj]), c='yellow', lw=1)

plt.legend()

x=np.linspace(Lambda[jj][0],Lambda[jj][-1],len(Lambda[jj]))
for i in range(len(Tablas[jj])):
    plt.axvline(x=Tablas[jj]['Lambda Vac'][i],c='dimgrey',lw=1.2,ls='--')
    plt.text(Tablas[jj]['Lambda Vac'][i] + 0.5, Tablas[jj]['Amplitude'][i] * 1.2,Tablas[jj]['Element'][i])
    if Tablas[jj]['Element'][i] == 'H I':
        plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']), c='DeepPink')
##        plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'], Tablas[jj][i]['Wavelength']), c='DeepPink')
    else:
        if Tablas[jj][i]['Sigma']!=0:
            plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']) , c='Lime')
##            plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']) , c='Lime')

#plt.savefig('F1.eps', format='eps', dpi=1000)


Using matplotlib backend: Qt4Agg


In [14]:
%matplotlib
kaksj = Modelos[jj][1](Lambda[jj])
lalskal = Modelos[jj][0](Lambda[jj])
specModel = spec+lalskal
plt.errorbar(Lambda[jj],Spectra[jj] ,SpectraIvar[jj],  c='k',lw=2,label='Data')   #Flujo en negro
plt.plot(Lambda[jj], specModel, lw=2, label='reducedSpectra')

Using matplotlib backend: Qt4Agg


In [14]:
%matplotlib

plt.figure(figsize=(16, 9))
for i in range(10):
    B=[]
    for j in range(len(Spectra[jj])):
        bb=SpectraIvar[jj][j]*random.uniform(-3,3)
        B.append(bb)
    plt.plot(Lambda[jj],(Spectra[jj] +B),c='orange')
plt.legend()

Using matplotlib backend: Qt4Agg


/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_axes.py:475: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


###   STAR SPECTRUM WITH A BLACKBODY SPECTRUM

In [41]:
#   star spectrum

theorFlux = blackbody.blackbody_lambda(Lambda[jj], 6000)
theorFlux = theorFlux/np.max(theorFlux)
theorFlux = theorFlux*np.max(Spectra[jj])
%matplotlib
#plt.plot(Lambda[jj], theorFlux)
plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')

In [28]:
ListaLineas=Table.read('LinesAll.txt', format='ascii')
ListaLineas

LAMBDA VAC ANG,SPECTRUM,TT,TERM,J J,LEVEL ENERGY CM 1
float64,string56,string16,string56,string56,string176
3727.092,[O II],E2,4So-2Do,3/2-3/2,0.00 - 26830.57
3729.875,[O II],E2,4So-2Do,3/2-5/2,0.00 - 26810.55
4341.684,H I,E1,2-5,*-*,82259.11 - 105291.66
4364.436,[O III],E2,1D-1S,2-0,20273.27 - 43185.74
4862.683,H I,E1,2-4,*-*,82259.11 - 102823.90
4932.603,[O III],E2,3P-1D,0-2,0.00 - 20273.27
4960.295,[O III],M1,3P-1D,1-2,113.18 - 20273.27
5008.24,[O III],M1,3P-1D,2-2,306.17 - 20273.27
5756.24,[N II],E2,1D-1S,2-0,15316.19 - 32688.64


In [100]:
#box_kernel = Box1DKernel(10)
#smoothed_data_box = convolve(Spectra[jj], box_kernel)
jj=0
gauss_kernel = Gaussian1DKernel(2)
smoothed_data_gauss = convolve(Spectra[jj], gauss_kernel)


lPPP = lineProfile(0,smoothed_data_gauss, Lambda[0],ListaLineas,Spectra, limResolucion)
lPPP

(array([42, 43]),)
31 56
(array([45, 46, 47]),)
31 56
(array([705, 706]),)
697 715
(array([728, 729]),)
715 742
(array([1197, 1198]),)
1187 1210
(array([1259, 1260]),)
1253 1268
(array([1284, 1285]),)
1268 1294
(array([1326]),)
1313 1344
(array([1930, 1931]),)
1926 1938
(array([2477, 2478]),)
2469 2492
(array([2491, 2492]),)
2469 2501
(array([2501]),)
2493 2510
(array([2515]),)
2510 2532
(array([2602]),)
2593 2607
(array([2611]),)
2607 2622
(array([2324]),)
2315 2329


lambda,inf,sup
int32,int32,int32
45,31,56
45,31,56
706,697,715
728,715,742
1198,1187,1210
1254,1253,1268
1284,1268,1294
1326,1313,1344
1932,1926,1938


In [102]:

jj = 0



%matplotlib
#plt.plot(Lambda[jj], theorFlux)
plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
plt.plot(Lambda[jj],smoothed_data_gauss, lw=2, c = 'cyan', label='Spectra')
for iasiao in range(len(lPPP)):
    plt.plot(Lambda[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]],Spectra[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]], lw=2.0, c = 'b')

Using matplotlib backend: Qt4Agg


In [19]:
#np.std(Spectra[0][])
jj = 0
spssp = [100]*len(Spectra[0])
divisi = 2
lambdDiv = int(mt.floor(len(spssp)/divisi))
lambdDivArr = [lambdDiv*a for a in range(divisi)]
lambdDivArr[-1] = len(Spectra[0])
for aa in range(len(lambdDivArr)-1):
    stdSpDiv = np.std(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #box_kernel = Box1DKernel(stdSpDiv)
    #smoothed_data_box = convolve(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]], box_kernel)
    
    gauss_kernel = Gaussian1DKernel(stdSpDiv)
    smoothed_data_gauss = convolve(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]], gauss_kernel)
    
    spssp[lambdDivArr[aa]:lambdDivArr[aa+1]] = smoothed_data_gauss # smoothed_data_box
    #meanSpDiv = np.mean(SpectraIvar[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #print "std: ", , "err: ", np.mean(SpectraIvar[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #print Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]]


%matplotlib
#plt.plot(Lambda[jj], theorFlux)
plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
plt.plot(Lambda[jj],spssp, lw=2, c = 'cyan', label='Spectra')

Using matplotlib backend: Qt4Agg


In [60]:
len(lambdDivArr)

100

###   Read tables Tabla... from files stored in resultTables and then density temperature calculation

In [40]:
#  From files in this path directory

#   This reads the tables "Tabla..." and obtains the best fitting by the difference of
#      wavelenght after fitting and the theoretical only the lines given in aarr. A table resultListDT is
#      built and then next module can be modified to read it.

resultListDT=Table(names=('ObjID','ra','dec','z', 'isFits'),dtype=('S33', 'f12', 'f12','f8', 'i1'))
resultList = Table.read("resultList.tbl", format = 'ascii')
limDim = 0.5
aarr = [3, 6, 7, 10, 12, 13, 14, 0, 1]
Tablas = []
for objID in resultList:
    nameTable = "Tabla"+str(objID[0])+".tbl"
    tableObjID = Table.read('resultTables/'+nameTable, format = 'ascii')
    conta = 0
    for iLine in aarr: # range(len(tableObjID)):
        if (mt.fabs(tableObjID['Lambda Vac'][iLine]-tableObjID['Wavelength'][iLine]) > limDim) or (tableObjID['Flux'][iLine]==0.0) :
            continue
        else:
            conta = conta + 1
    if conta == len(aarr):
        resultListDT.add_row([objID[0], 0, 0, 0, 1])
        Tablas.append(tableObjID)

ion = [' [OIII]', ' [SII]', ' [NII]', ' [OII]', ' [NII]', ' [SII]', ' [OIII]', ' [OII]']
for i in range(1,5):
    colT= Column(name='Temp_'+str(i)+ion[2*(i-1)], length=len(resultListDT),unit=u.K)
    colD= Column(name='Den_'+str(i)+ion[2*(i-1)+1], length=len(resultListDT),unit=1 / (u.cm**3))
    resultListDT.add_columns([colT,colD])
resultListDT

#   it obtains the electron density and temperature by the PyNeb cross iterative method for all couple of given species.

fig = plt.figure()
fig.suptitle('')
ax = fig.add_subplot(111)
ax.set_ylabel('Density $n_e (cm^{-3})$')
ax.set_xlabel('Temperature $T_e (K)$')

for jj in [11]:  #  range(len(Tablas)):
    k=0
    #if Lambda[jj][0] > 3720.0 :
    #    k=2
    try:
        a=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        b=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
        c=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        d=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
    except ValueError:
        continue

    for i in IntrinsicHB:
        rc.setCorr((Tablas[jj]['Flux'][11-k]/Tablas[jj]['Flux'][4-k])/i, Tablas[jj]['Lambda Vac'][11-k], Tablas[jj]['Lambda Vac'][4-k])
        try:
            a2=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            b2=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
            c2=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])),Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            d2=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
        except ValueError:
            continue
        if b2 != () or d2 != ():
            plt.scatter(b2[0], b2[1],c='c')
            plt.scatter(d2[0], d2[1],c='c')
        if i == 2.86:
            resultListDT['Temp_1 [OIII]'][jj]=a2[0]
            resultListDT['Den_1 [SII]'][jj]=a2[1]
            resultListDT['Temp_3 [NII]'][jj]=c2[0]
            resultListDT['Den_3 [SII]'][jj]=c2[1]
            if b2 != () or d2 != ():
                resultListDT['Temp_2 [NII]'][jj]=b2[0]
                resultListDT['Den_2 [OII]'][jj]=b2[1] 
                resultListDT['Temp_4 [OIII]'][jj]=d2[0]
                resultListDT['Den_4 [OII]'][jj]=d2[1]
            #if Lambda[jj][0] > 3720.0 :
            #    resultListDT['Temp_2 [NII]'][jj]=0.0
            #    resultListDT['Den_2 [OII]'][jj]=0.0
            #    resultListDT['Temp_4 [OIII]'][jj]=0.0
            #    resultListDT['Den_4 [OII]'][jj]=0.0
    #resultListDT.write('resultTables/TemDenGalaxies'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
resultListDT


ObjID,ra,dec,z,isFits,Temp_1 [OIII],Den_1 [SII],Temp_2 [NII],Den_2 [OII],Temp_3 [NII],Den_3 [SII],Temp_4 [OIII],Den_4 [OII]
,,,,,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3
string264,float32,float32,float64,int8,float64,float64,float64,float64,float64,float64,float64,float64
2MASX J00505919-0031294,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UGC 06315 NED02SP1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MRK 0168SP1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UM 213SP1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SDSS J011031.04+010200.7SP1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MRK 0738SP1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TON 0151SP2,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2MASX J15540451+2134564SP1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:

# Generate fake data
np.random.seed(0)
x = np.linspace(-10., 5., 200)
y = 30 * np.exp(-0.5 * (x - (1.3))**2 / 0.8**2)
y1 = 40 * np.exp(-0.5 * (x - (1.0))**2 / 0.8**2)
y += np.random.normal(0., 0.2, x.shape)
y1 += np.random.normal(0., 0.2, x.shape)
y = y+y1

# Fit the data using a Gaussian
g_init = models.Gaussian1D(amplitude=1.5, mean=1.3, stddev=1.)
g_init2 = models.Gaussian1D(amplitude=1., mean=1.0, stddev=1.)

jointpars = { 'g_init':[g_init.parameters], 'g_init2': [g_init2.parameters]}

pars = { g_init: 'g_init', g_init2: 'g_init2'}
fit_g = fitting.JointFitter( pars , jointpars, [30, 1.3, 1.0])   # LevMarLSQFitter()
g = fit_g(g_init, g_init2, x, y)

# Plot the data with the best-fit model


TypeError: 1 parameter(s) provided but 3 expected

In [55]:
%matplotlib
plt.figure(figsize=(8,5))
plt.plot(x, y, 'ko')
plt.plot(x, t(x), 'b-', lw=2, label='Trapezoid')
plt.plot(x, g(x), 'r-', lw=2, label='Gaussian')
plt.plot(x, fit_g.fit_info['fjac'][3], 'g-', lw=2, label='Gaussian')

plt.xlabel('Position')
plt.ylabel('Flux')
plt.legend(loc=2)

Using matplotlib backend: Qt4Agg


KeyError: 'fjac'

In [63]:
#print fit_g.fit_info
#print g
g_init.parameters


array([ 1.5,  1.3,  1. ])

In [ ]:
nfev': None,
                         'fvec': None,
                         'fjac': None,
                         'ipvt': None,
                         'qtf': None,
                         'message': None,
                         'ierr': None,
                         'param_jac': None,
                         'param_cov': None

In [2]:
#   a table resultListDT relates the name of the objects and the resulting temoerature and density
#   for the four couples of species

resultListDT=Table.read('resultListDT.tbl', format='ascii')

#   Obtain the lists Spectra, Headers, spectrum variance of the intensity according with a list of galaxy names
#   and a Table Tabla of models for each line for all the spectra

Headers = []
Spectra = []
Spectra2 = []
SpectraCont = []
Lambda = []
Modelos=[]
Tablas=[]
HDUs=[]
TypeFITS=[]
SpectraIvar = []
LINEProfile = []
paramCont=0.4
typeFITS =0
for x in range(len(resultListDT['ObjID'])):
    try:
        name = resultListDT['ObjID'][x]
        z=resultListDT['z'][x]
        isFits = resultListDT['isFits'][x]
        if isFits:
            hdulist = fits.open('Exam_fits/'+name+'.fit')
            head= hdulist[0].header
            typeFITS=0
            if head['NAXIS'] == 0:
                total= hdulist[1].data
            elif head['NAXIS'] == 1:
                typeFITS=1
            if typeFITS==0:
                spectraFlux = total['flux']
                spectraIvar = total['ivar']
                linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
                lambd = linlam/(1+z)
            else:
                spectraFlux = hdulist[0].data #*(1e-19)
                spectraIvar = spectraFlux*0
                if 'CDELT1' in head:
                    lambd = np.linspace(head['CRVAL1'], head['CRVAL1']+head['CDELT1']*(len(spectraFlux)-1), len(spectraFlux))
                elif 'W0' in head:
                    lambd = np.linspace(head['W0'], head['W0']+head['WPC']*(len(spectraFlux)-1), len(spectraFlux))
                else:
                    print "Can't identify FITS format in file: ", str(name)
                    break
                lambd = lambd/(1+z)
        else:
            specTable = Table.read('Exam_fits/'+str(name)+'.txt', format='ascii')
            spectraFlux = specTable['Intensity'].data #.tolist()
            spectraIvar = spectraFlux*0
            lambd = specTable['Spectral-axis'].data #.tolist()
            lambd = lambd/(1+z)
        orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)

        TypeFITS.append(typeFITS)
        Spectra2.append(spectraFlux)
        print "Object [",x,"]: ", name
        spec =np.array(spectraFlux)
        spec2, lambd = r(spec, lambd, paramCont*orderU)
        
        #gauss_kernel = Gaussian1DKernel(2)
        #smoothed_data_gauss = convolve(spectraFlux, gauss_kernel)
        #spec2 = smoothed_data_gauss

        limResolucion = mt.ceil(lambd[1]-lambd[0])

    # ..............    

        if lambd[0] > 3720.0 :
            print "The object isn't in expected spectrum region (3700 - 6800 Angstrom)"
            ListaLineas=Table.read('LinesPrincipal.txt', format='ascii')
            LS=np.zeros([13,10])

            lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra2, limResolucion)
            LINEProfile.append(lineProf)
            # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
            # que es mas preciso al determinar la amplitud del modelo gaussiano
            for ina in range(len(lineProf)):
                lInf = lineProf[ina]['inf']
                lSup = lineProf[ina]['sup']
                lDel = lSup-lInf
                if (lInf-3*lDel) >= 0 :
                    lInf = lInf-3*lDel
                if (lSup+3*lDel) <= len(lambd) :
                    lSup = lSup+3*lDel
                m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
                lineLambd = lambd[lInf:lSup]
                lineIntCont = lines( lineLambd , m1, b1)
                spec2[lInf:lSup] = lineIntCont
            ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra2, spectraIvar,2000, limResolucion)
    #        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)
        else:
            # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
            # que es mas preciso al determinar la amplitud del modelo gaussiano
            ListaLineas=Table.read('LinesAll.txt', format='ascii')
            lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra2, limResolucion)
            LINEProfile.append(lineProf)
            for ina in range(len(lineProf)):
                lInf = lineProf[ina]['inf']
                lSup = lineProf[ina]['sup']
                lDel = lSup-lInf
                if (lInf-3*lDel) >= 0 :
                    lInf = lInf-3*lDel
                if (lSup+3*lDel) <= len(lambd) :
                    lSup = lSup+3*lDel
                m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
                lineLambd = lambd[lInf:lSup]
                lineIntCont = lines( lineLambd , m1, b1)
                spec2[lInf:lSup] = lineIntCont
            LS=np.zeros([len(ListaLineas),10])
            ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra2, spectraIvar,2000, limResolucion)
    #        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)

        Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','Flux_Err', 'FWHM', 'eqWidth', 'S/N>3'),
                    dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8','S5'))
        Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
        Tabla['Element']=ListaLineas['SPECTRUM']
        AA=split_list(ModelG1[0].parameters, wanted_parts=len(ListaLineas))
        AAA=split_list(ModelG1[1].parameters, wanted_parts=2)
    #    AA=split_list(ModelL1[0].parameters, wanted_parts=len(ListaLineas))
    #    AAA=split_list(ModelL1[1].parameters, wanted_parts=2)
        for ii in range(len(ListaLineas)):
            if 3*np.mean(spectraIvar[lineProf[0]['inf']:lineProf[0]['sup']]) >= AA[ii][0]:
                Tabla['S/N>3'][ii] = str(False)
            else:
                Tabla['S/N>3'][ii] = str(True)
            if AA[ii][2]<0:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], mt.fabs(AA[ii][2])])
            else:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], AA[ii][2]])
            if ii==0 or ii==1:
                if AAA[ii][2]<0:
                    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], mt.fabs(AAA[ii][2])])
                else:
                    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], AAA[ii][2]])
        Modelos.append(ModelG1)
        #    Modelos.append(ModelL1)
        #print Tabla 
        Tablas.append(Tabla)
        SpectraIvar.append(spectraIvar)
        SpectraCont.append(spec2)
        Lambda.append(lambd)
        Spectra.append(spectraFlux)
    except ValueError:
        print "It has not been possible to process the spectrum of: ", name

NameError: name 'total' is not defined

###   Star spectrum collaboration with Univ. Pasto

In [6]:
resultListDT=Table.read('newObjSDSSFound.tbl', format='ascii')
name = resultListDT['ObjID'][0]
hdulist = fits.open('Exam_fits/'+name+'.fit')
head= hdulist[0].header
#total= hdulist[0].data
lambd = np.linspace(0, 1000, len(total[0]))
%matplotlib
plt.imshow(total, cmap='gray')#, vmin=-30, vmax=150)


Using matplotlib backend: Qt4Agg


TypeError: Image data can not convert to float

In [7]:
total

FITS_rec([(15.1883, 3.5822001, 0.097029231, 0, 0, 1.2656289, 8.6483488, 13.25177),
       (14.52297, 3.5822999, 0.097941689, 0, 0, 1.2651138, 7.2937446, 12.101864),
       (21.377337, 3.5824001, 0.08913774, 0, 0, 1.2640908, 6.7241831, 17.5219),
       ...,
       (211.31332, 3.9628999, 0.057048395, 0, 0, 0.6769467, 4.4245758, 208.33597),
       (213.42195, 3.9630001, 0.056964647, 0, 0, 0.67687726, 4.3627143, 207.58827),
       (212.33568, 3.9631, 0.056582704, 0, 0, 0.67683941, 4.4938502, 208.03232)], 
      dtype=[('flux', '>f4'), ('loglam', '>f4'), ('ivar', '>f4'), ('and_mask', '>i4'), ('or_mask', '>i4'), ('wdisp', '>f4'), ('sky', '>f4'), ('model', '>f4')])

In [63]:
#total
#resultListDT
#head

#box_kernel = Box1DKernel(10)
#smoothed_data_box = convolve(Spectra[jj], box_kernel)
gauss_kernel = Gaussian1DKernel(20)
index = 1065
smoothed_data_gauss = convolve(total[:,index], gauss_kernel)

%matplotlib
#plt.imshow(total[:][327:1060], cmap='gray')#, vmin=-30, vmax=150)
plt.plot(lambd, total[:,index], 'k')
plt.plot(lambd, smoothed_data_gauss, 'cyan', lw=2.0)



Using matplotlib backend: Qt4Agg


###    Find the smoothed spectrum of a star

In [81]:
resultListDT=Table.read('newObjSDSSFound.tbl', format='ascii')
name = resultListDT['ObjID'][0]
hdulist = fits.open('Exam_fits/'+name+'.fits')
head= hdulist[0].header
total= hdulist[1].data
linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
lambd = linlam/(1+0.0001)
spectraFlux = total['flux']
Spectra = []
Spectra.append(spectraFlux)
Lambda=[]
Lambda.append(lambd)
ListaLineas=Table.read('LinesStar.txt', format='ascii')
lima = Lambda[jj][1]-Lambda[jj][0]

In [48]:
#box_kernel = Box1DKernel(10)
#smoothed_data_box = convolve(Spectra[jj], box_kernel)
gauss_kernel = Gaussian1DKernel(15)
smoothed_data_gauss = convolve(spectraFlux, gauss_kernel, boundary=u'extend')

%matplotlib
plt.plot(lambd, spectraFlux, 'k')
plt.plot(lambd, smoothed_data_gauss, 'cyan', lw=2.0)



Using matplotlib backend: Qt4Agg


In [82]:
jj = 0
spssp = [100]*len(Spectra[0])
divisi = 10
lambdDiv = int(mt.floor(len(spssp)/divisi))
lambdDivArr = [lambdDiv*a for a in range(divisi)]
lambdDivArr[-1] = len(Spectra[0])
SG = []
for aa in range(len(lambdDivArr)-1):
    stdSpDiv = np.std(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #box_kernel = Box1DKernel(stdSpDiv)
    #smoothed_data_box = convolve(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]], box_kernel)
    #print stdSpDiv
    gauss_kernel = Gaussian1DKernel(0.05*stdSpDiv)
    meanSpDiv = np.mean(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    smoothed_data_gauss = convolve(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]], gauss_kernel, boundary=u'extend',fill_value=meanSpDiv, normalize_kernel=True)
    SG.append(smoothed_data_gauss)
    spssp[lambdDivArr[aa]:lambdDivArr[aa+1]] = smoothed_data_gauss # smoothed_data_box
    #print "std: ", , "err: ", np.mean(SpectraIvar[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #print Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]]

#stdSp = np.std(Spectra[0])
#STD = [stdSp]*len(Spectra[0])
#spssp = spssp+STD

In [83]:
def lineProfile2(i,spec, lambd,ListLines,ListGal, limResoluc):
    linePr = Table(names=('lambda', 'inf', 'sup'), dtype=('i5','i5', 'i5'))
    for c in range(0, len(ListLines)):
        lambdLine=ListLines['LAMBDA VAC ANG'][c]          #Líneas obtenidas de la base de datos Astroquery
        v = np.where((lambd >= lambdLine-limResoluc) & (lambd <= lambdLine+limResoluc))
        print v
        if len(v[0]) == 0:
            ind = 0
            l_inf = 0
            l_sup = 0
            iM=0
        else:
            ind = 1
            
            l_inf = v[0][0]-ind
            l_sup = v[0][-1]+ind
            slopeRight = spec[l_sup+1]-spec[l_sup]
            cooc = 0
            while cooc<1:
                l_sup=l_sup+1
                sloRaux = slopeRight
                slopeRight = spec[l_sup+1]-spec[l_sup]
                if slopeRight*sloRaux <= 0:
                    cooc = cooc + 1
            cooc = 0
            slopeLeft = spec[l_inf-1]-spec[l_inf]
            while cooc<1:
                l_inf=l_inf-1
                sloLaux = slopeLeft
                slopeLeft = spec[l_inf-1]-spec[l_inf]
                if slopeLeft*sloLaux <= 0:
                    cooc = cooc + 1
            print l_inf,l_sup
            l_sup=l_sup+1
            # Una vez encontrado el intervalo se busca el indice donde
            # tiene el máximo valor de intensidad
            indMaxInt = np.where(ListGal[i][l_inf:l_sup]==np.min(ListGal[i][l_inf:l_sup]))
            indMax=indMaxInt[0][0]+l_inf
            iM = indMax
            
        linePr.add_row((iM, l_inf, l_sup))        
    return linePr

In [84]:
kLineP = lineProfile2(0, spssp, Lambda[0], ListaLineas, Spectra, lima)
kLineP

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([554, 555, 556]),)
548 562
(array([576, 577, 578, 579]),)
567 598
(array([1046, 1047, 1048]),)
1044 1052
(array([1108, 1109, 1110]),)
1099 1112
(array([1132, 1133, 1134, 1135]),)
1125 1142
(array([1174, 1175, 1176]),)
1172 1178
(array([1779, 1780, 1781]),)
1774 1783
(array([2326, 2327, 2328]),)
2324 2330
(array([2340, 2341]),)
2334 2343
(array([2350, 2351]),)
2348 2354
(array([2364, 2365]),)
2361 2367
(array([2450, 2451, 2452]),)
2448 2454
(array([2460, 2461]),)
2458 2463
(array([2173, 2174]),)
2170 2179
(array([1882, 1883]),)
1867 1895
(array([3472, 3473]),)
3469 3477
(array([3495, 3496]),)
3487 3501
(array([3556, 3557]),)
3551 3560


lambda,inf,sup
int32,int32,int32
0,0,0
0,0,0
554,548,563
598,567,599
1047,1044,1053
1099,1099,1113
1132,1125,1143
1175,1172,1179
1779,1774,1784


In [91]:
%matplotlib
aa = 1
#plt.plot(Lambda[jj], theorFlux)
plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
#plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
plt.plot(Lambda[jj],spssp, lw=2, c = 'cyan', label='Spectra')
#plt.plot(Lambda[jj][lambdDivArr[aa]:lambdDivArr[aa+1]],SG[aa], lw=2, c = 'cyan', label='Spectra')
#plt.plot(Lambda[jj][lambdDivArr[aa]:lambdDivArr[aa+1]],Spectra[jj][lambdDivArr[aa]:lambdDivArr[aa+1]], lw=2, c = 'g', label='Spectra')
plt.plot(Lambda[jj],spec2, lw=2, c = 'g', label='Spectra')
lPPP = kLineP
plt.plot(Lambda[jj],Spectra[jj]-spssp, lw=0.5, c = 'gray', label='Spectra')
for iasiao in [17]: # range(len(lPPP)):
    plt.plot(Lambda[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]],Spectra[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]], lw=2.0, c = 'b')


Using matplotlib backend: Qt4Agg


In [86]:
spec2 = spssp
for ina in range(len(kLineP)):
    lInf = kLineP[ina]['inf']
    lSup = kLineP[ina]['sup']
    lDel = lSup-lInf
    if (lInf-1*lDel) >= 0 :
        lInf = lInf-1*lDel
    if (lSup+1*lDel) <= len(lambd) :
        lSup = lSup+1*lDel
    m1, b1 = linModel( Lambda[jj][lInf] , spec2[lInf] , Lambda[jj][lSup], spec2[lSup])
    lineLambd = Lambda[jj][lInf:lSup]
    lineIntCont = lines( lineLambd , m1, b1)
    spec2[lInf:lSup] = lineIntCont